## Imports

### Python modules

In [98]:
import pandas as pd
import numpy as np
from datetime import datetime

## Data Processing

### Dataframe Creation

Read target csv to a Dataframe, ignore all bad lines. Python will tell me which lines are ignored for manual cleanup. Only seems to happen when boards were rebooted - no data loss, easy to fix.

In [110]:
data = pd.read_csv('./SRAM3/1/half/nvRAM_data_02-06-2021-1050.csv', error_bad_lines=False)
log_length = data.shape[0]

### Neutron Logs

Get first/last log timestamps into datetime objects

In [123]:
exp_tformat = '%Y-%m-%d_%H-%M-%S-%f'
exp_firststamp = datetime.strptime(data.iloc[0]['Timestamp'], exp_tformat)
exp_laststamp = datetime.strptime(data.iloc[log_length-1]['Timestamp'], exp_tformat)

Create neutron log names from datetime objects, import neutron logs ignoring first line

In [148]:
beamlog_fname_tformat = '%Y-%m-%d'
beamlog_firstday = datetime.strftime(exp_firststamp, beamlog_tformat)
beamlog_lastday = datetime.strftime(exp_laststamp, beamlog_tformat)
beamlog_first = pd.read_csv(f"../neutrons/countlog-{beamlog_firstday}.txt", delim_whitespace=True, header=None, skiprows=1)
beamlog_last = pd.read_csv(f"../neutrons/countlog-{beamlog_lastday}.txt", delim_whitespace=True, header=None, skiprows=1)

### Identifying Runs

Create a list of every instance of M,A,WRCHCKBRD. This log line indicates the start of every new experiment run. Should probably make this a user input to this Python so that different experiment setups are easily accounted for.

Create a list of every instance of M,A,INIT. Indicates start of board power cycle.

This method is the easiest, but will label all INIT cycles as a separate experimental run. No loss of data occurs so this is ok.

In [54]:
a = data.loc[(data['Board'] == 'M') & (data['Mikroe_socket'] == 'A') & (data['Status'] == 'WRCHCKBRD')].index.tolist()
b = data.loc[(data['Board'] == 'M') & (data['Mikroe_socket'] == 'A') & (data['Status'] == 'INIT')].index.tolist()

Combine both lists, and append new list with final log line number.

In [55]:
c = a + b
c = sorted(c)
c.append(log_length)

For each element n in a, cycle through all the integers between n and n+1, and add a run ID number to a blank list. When all the integers have been cycled through, do the same for all integers between n+1 and n+2, and with run ID+1. Continue until all log lines are associated with a run ID.

In [56]:
j = 0
k = 1
run_id = 1
run_col = []
for h in range(1, len(c)):
    for i in range(c[j], c[k]):
        run_col.append(run_id)
    j += 1
    k += 1
    run_id += 1

In [57]:
data.insert(loc=0, column='Run_ID', value=run_col)

In [58]:
data[600:660]

,Run_ID,Timestamp,Board,Mikroe_socket,Status,SDC_val,SDC_loc
600,10,2021-06-02_10-53-17-818549,M,B,SDC,FB,A0
601,10,2021-06-02_10-53-17-834564,M,B,SDC,FF,A1
602,10,2021-06-02_10-53-17-850536,M,B,SDC,FF,A2
603,10,2021-06-02_10-53-17-866564,M,B,SDC,F7,A3
604,10,2021-06-02_10-53-17-882528,M,B,SDC,FF,A4
605,10,2021-06-02_10-53-17-898551,M,B,SDC,F1,A5
606,10,2021-06-02_10-53-17-914536,M,B,SDC,7F,A6
607,10,2021-06-02_10-53-17-930545,M,B,SDC,FD,A7
608,10,2021-06-02_10-53-17-946544,M,B,SDC,FF,A8
609,10,2021-06-02_10-53-17-962528,M,B,SDC,FF,A9


### Hot and Cold Runs

Format datetime object into neutron log timestamps

In [149]:
beamlog_stamp_tformat = '%d/%m/%Y %H:%M:%S'
beamlog_firststamp = datetime.strftime(exp_firststamp, beamlog_stamp_tformat)
beamlog_laststamp = datetime.strftime(exp_laststamp, beamlog_stamp_tformat)


Remake neutron log column names

In [150]:
beamlog_new_colnames = ["Date", "HMS_time", "Millisecs", "Count1", "Count2", "Count3", "Count4", "unknown", "Beam_current"]
beamlog_first.columns = beamlog_new_colnames
beamlog_last.columns = beamlog_new_colnames

### How many bits tend to get flipped?

Get occurrences of each type of error

In [65]:
num_see = data.loc[(data['Status'] == 'SDC')].index
see_types_hex = see.value_counts().index.tolist()
see_freqs = see.value_counts().tolist()
see_bins = list(zip(see_types_hex, see_freqs))

Use this for Hamming distance between errors and golden value:

In [83]:
xortest1 = 169
xortest2 = 170

bin(xortest1 ^ xortest2).count('1')

2